# Model Training

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import importlib
import os
import sys
# module = os.path.abspath('/home/learner/DLA_project/src/main')
module = os.path.abspath("C:/Users\\18145\\development\\wesad_experiments\\src\\main")
if module not in sys.path:
    sys.path.append(module)
from DataManager import DataManager

In [2]:
manager = DataManager()

In [3]:

manager.load_all()


Loading data from S2
	Path=C:\WESAD\S2\S2.pkl
Loading data from S3
	Path=C:\WESAD\S3\S3.pkl
Loading data from S4
	Path=C:\WESAD\S4\S4.pkl
Loading data from S5
	Path=C:\WESAD\S5\S5.pkl
Loading data from S6
	Path=C:\WESAD\S6\S6.pkl
Loading data from S7
	Path=C:\WESAD\S7\S7.pkl
Loading data from S8
	Path=C:\WESAD\S8\S8.pkl
Loading data from S9
	Path=C:\WESAD\S9\S9.pkl
Loading data from S10
	Path=C:\WESAD\S10\S10.pkl
Loading data from S11
	Path=C:\WESAD\S11\S11.pkl


In [4]:
manager.compute_features();
manager.compute_features_stress();

conputing features..
Considering subject  2
Considering subject  3
Considering subject  4
Considering subject  5
Considering subject  6
Considering subject  7
Considering subject  8
Considering subject  9
Considering subject  10
Considering subject  11
conputing features..
Considering subject  2
Considering subject  3
Considering subject  4
Considering subject  5
Considering subject  6
Considering subject  7
Considering subject  8
Considering subject  9
Considering subject  10
Considering subject  11


In [77]:
print("We have", len(manager.SUBJECTS), " subjects")

for feature in manager.FEATURES.keys():
    print("there are ", len(manager.FEATURES[feature]), " values for ", feature)
    print(manager.FEATURES[feature][3277:3288])

We have 10  subjects
there are  44461  values for  a_mean
[0.6426000595092773, 0.6426000595092773, 0.6426000595092773, 0.6426000595092773, 0.6426000595092773, 0.6426000595092773, 0.6426000595092773, 0.6426000595092773, 0.6426000595092773, 0.6426000595092773, 0.6426000595092773]
there are  44461  values for  a_std
[-0.04559999704360962, -0.04559999704360962, -0.04559999704360962, -0.04559999704360962, -0.04559999704360962, -0.04559999704360962, -0.04559999704360962, -0.04559999704360962, -0.04559999704360962, -0.04559999704360962, -0.04559999704360962]
there are  44461  values for  a_maxx
[-0.6898000240325928, -0.6898000240325928, -0.6898000240325928, -0.6898000240325928, -0.6898000240325928, -0.6898000240325928, -0.6898000240325928, -0.6898000240325928, -0.6898000240325928, -0.6898000240325928, -0.6898000240325928]
there are  44461  values for  a_maxy
[-0.1681766395512081, -0.16820373953666, -0.1682159157196681, -0.16822052995931536, -0.16817386801469902, -0.16810921560015002, -0.16804

In [102]:
# Lets go ahead and reshape this data such that we have
# samples, features = [N, 15]

X1 = []
X2 = []
for i in range(0, ) 


X1 = np.hstack([manager.FEATURES['a_mean'], manager.FEATURES['a_std'], manager.FEATURES['a_maxx'], manager.FEATURES['a_maxy'], manager.FEATURES['a_maxz'],\
                  manager.FEATURES['e_max'], manager.FEATURES['e_min'], manager.FEATURES['e_mean'], manager.FEATURES['e_range'], manager.FEATURES['e_std'],\
                  manager.FEATURES['t_max'], manager.FEATURES['t_min'], manager.FEATURES['t_mean'], manager.FEATURES['t_range'], manager.FEATURES['t_std']] )
print(np.shape(X1))

# X1 = np.reshape(X1, (len(manager.FEATURES['a_mean']), 15))

print(np.shape(X1))


X2 = np.hstack([manager.STRESS_FEATURES['a_mean'], manager.STRESS_FEATURES['a_std'], manager.STRESS_FEATURES['a_maxx'], manager.STRESS_FEATURES['a_maxy'], manager.STRESS_FEATURES['a_maxz'],\
                  manager.STRESS_FEATURES['e_max'], manager.STRESS_FEATURES['e_min'], manager.STRESS_FEATURES['e_mean'], manager.STRESS_FEATURES['e_range'], manager.STRESS_FEATURES['e_std'],\
                  manager.STRESS_FEATURES['t_max'], manager.STRESS_FEATURES['t_min'], manager.STRESS_FEATURES['t_mean'], manager.STRESS_FEATURES['t_range'], manager.STRESS_FEATURES['t_std']] )
print(np.shape(X1))

# X2 = np.reshape(X2, (len(manager.STRESS_FEATURES['a_mean']), 15))

print(np.shape(X2))

(44461, 15)
(23780, 15)


[[1.14540005 1.14540005 1.14540005 1.14540005 1.14540005 1.14540005
  1.14540005 1.14540005 1.14540005 1.14540005 1.14540005 1.14540005
  1.14540005 1.14540005 1.14540005]
 [1.14540005 1.14540005 1.14540005 1.14540005 1.14540005 1.14540005
  1.14540005 1.14540005 1.14540005 1.14540005 1.14540005 1.14540005
  1.14540005 1.14540005 1.14540005]
 [1.14540005 1.14540005 1.14540005 1.14540005 1.14540005 1.14540005
  1.14540005 1.14540005 1.14540005 1.14540005 1.14540005 1.14540005
  1.14540005 1.14540005 1.14540005]
 [1.14540005 1.14540005 1.14540005 1.14540005 1.14540005 1.14540005
  1.14540005 1.14540005 1.14540005 1.14540005 1.14540005 1.14540005
  1.14540005 1.14540005 1.14540005]
 [1.14540005 1.14540005 1.14540005 1.14540005 1.14540005 1.14540005
  1.14540005 1.14540005 1.14540005 1.14540005 1.14540005 1.14540005
  1.14540005 1.14540005 1.14540005]
 [1.14540005 1.14540005 1.14540005 1.14540005 1.14540005 1.14540005
  1.14540005 1.14540005 1.14540005 1.14540005 1.14540005 1.14540005
  1.

In [74]:
print(manager.FEATURES['a_mean'][100:110])

print(manager.FEATURES['a_std'][400:410])

print(manager.FEATURES['a_maxx'][0:10])

[1.145400047302246, 1.145400047302246, 1.145400047302246, 1.145400047302246, 1.145400047302246, 1.145400047302246, 1.145400047302246, 1.145400047302246, 1.145400047302246, 1.145400047302246]
[-0.06319999694824219, -0.06319999694824219, -0.06319999694824219, -0.06319999694824219, -0.06319999694824219, -0.06319999694824219, -0.06319999694824219, -0.06319999694824219, -0.06319999694824219, -0.06319999694824219]
[0.6109999418258667, 0.6109999418258667, 0.6109999418258667, 0.6109999418258667, 0.6109999418258667, 0.6109999418258667, 0.6109999418258667, 0.6109999418258667, 0.6109999418258667, 0.6109999418258667]


In [7]:
# import pip

# try:
#     import keras
# except ImportError:
#     ! sudo pip3 install keras
# try:
#     import h5py
# except ImportError:
#     ! sudo pip3 install h5py
# try:
#     import ibmiotf
# except ImportError:
#     ! sudo pip3 install ibmiotf
# try:
#     import tensorflow
# except ImportError:
#     ! sudo pip3 install tensorflow

In [8]:
import numpy as np
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
import sklearn
from  sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
import tensorflow as tf
import keras
from keras.preprocessing import sequence

from keras.models import Sequential

from keras.layers import Dense, Embedding, Flatten

from keras.layers import LSTM

Using TensorFlow backend.


In [9]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

session_conf = tf.ConfigProto(intra_op_parallelism_threads=8, inter_op_parallelism_threads=8, )

tf.set_random_seed(1)

session = tf.Session(graph=tf.get_default_graph(), config=session_conf)
keras.backend.set_session(session)

W0712 20:01:39.512037 18800 deprecation_wrapper.py:119] From C:\Users\18145\AppData\Roaming\Python\Python37\site-packages\keras\backend\tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0712 20:01:39.514793 18800 deprecation_wrapper.py:119] From C:\Users\18145\AppData\Roaming\Python\Python37\site-packages\keras\backend\tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0712 20:01:39.517055 18800 deprecation_wrapper.py:119] From C:\Users\18145\AppData\Roaming\Python\Python37\site-packages\keras\backend\tensorflow_backend.py:186: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0712 20:01:39.587185 18800 deprecation_wrapper.py:119] From C:\Users\18145\AppData\Roaming\Python\Python37\site-packages\keras\backend\tensorflow_backend.py:190: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables 

In [10]:
# x1 is the baseline data
# x2 is the stress data

y1 = [0] * len(X1)
y2 = [1] * len(X2)

In [11]:
print(len(y1))
print(len(y2))

44461
23780


Now we need to concat the data between baseline and stress so that we can split it into training and test sets

In [54]:
X = np.concatenate((X1, X2), axis=0)
print(np.shape(X))

y = np.concatenate((y1,y2), axis=0)
print(np.shape(y))

(68241, 15)
(68241,)


In [55]:
# Split the data up in train and test sets
from sklearn.model_selection import train_test_split

# since we don't have the same number of samples for x and y, 
# we will drop off some x values for creating the sample
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.25, random_state=42)

In [56]:
print(np.shape(X_train))
print(np.shape(X_test))
print(np.shape(y_train))
print(np.shape(y_test))

(51180, 15)
(17061, 15)
(51180,)
(17061,)


Need to scale the data and hope we get better results.
1. fit the scaler to the training data (fit_transform)
2. transform training data with the scaler
3. fit the model with transformed data
4. transform test data with the scaler
5. predict using model and output of (4)

It will be better to have our data scaled between 0 and 1, so lets go ahead and create a function to 
normalize the data that way now.

In [15]:
# def normalize(data):
#     scaler = MinMaxScaler(feature_range=(0, 1))
#     return scaler.fit_transform(data)
# X_train = normalize(X_train)
# X_test = normalize(X_test)

In [16]:
# Should be

# input_shape=(number of sequences=?, time_steps=None, features=15)

# target=(number of sequences, time_steps, targets)

In [57]:
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [58]:
print(np.shape(X_train))
print(np.shape(X_test))
print(np.shape(y_train))
print(np.shape(y_test))

(51180, 1, 15)
(17061, 1, 15)
(51180,)
(17061,)


In [19]:
num_neurons = 15
num_features = 15

In [20]:
print('Building the LSTM network...')


model = Sequential()

model.add(LSTM(num_neurons, input_shape=(1, num_features), return_sequences=True))
# model.add(LSTM(16, input_shape=(1, 15), dropout=0.35, recurrent_dropout=0.35, return_sequences=True))

# Note:
# Need to do return_sequences=False for the layer *before* dense
# https://stackoverflow.com/questions/51763983/error-when-checking-target-expected-dense-1-to-have-3-dimensions-but-got-array
model.add(LSTM(num_neurons, input_shape=(1, num_features), return_sequences=False))
model.add(Dense(1, activation='sigmoid'))


W0712 20:01:39.882449 18800 deprecation_wrapper.py:119] From C:\Users\18145\AppData\Roaming\Python\Python37\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



Building the LSTM network...


In [59]:
model.summary()

print("inputs: " , model.input_shape)

print("outputs: ", model.output_shape)

print("actual inputs: ", np.shape(X_train))

print("actual outputs: ", np.shape(y_train))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 1, 16)             2048      
_________________________________________________________________
lstm_2 (LSTM)                (None, 16)                2112      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 4,177
Trainable params: 4,177
Non-trainable params: 0
_________________________________________________________________
inputs:  (None, 1, 15)
outputs:  (None, 1)
actual inputs:  (51180, 1, 15)
actual outputs:  (51180,)


In [60]:
from keras.optimizers import SGD
opt = SGD(lr=0.05)
model.compile(loss='binary_crossentropy',

              optimizer=opt,

              metrics=['accuracy'])


In [61]:
print(np.shape(X_train))

print(X_train[0:10])



(51180, 1, 15)
[[[34.4163208  34.4163208  34.4163208  34.4163208  34.4163208
   34.4163208  34.4163208  34.4163208  34.4163208  34.4163208
   34.4163208  34.4163208  34.4163208  34.4163208  34.4163208 ]]

 [[ 0.30010986  0.30010986  0.30010986  0.30010986  0.30010986
    0.30010986  0.30010986  0.31079102  0.31079102  0.31079102
    0.31079102  0.31079102  0.31079102  0.31079102  0.31079102]]

 [[ 0.3452301   0.3452301   0.3452301   0.3452301   0.3452301
    0.3452301   0.3452301   0.3452301   0.3452301   0.3452301
    0.3452301   0.3452301   0.3452301   0.3452301   0.3452301 ]]

 [[ 0.64260006  0.64260006  0.64260006  0.64260006  0.64260006
    0.64260006  0.64260006  0.64260006  0.64260006  0.64260006
    0.64260006  0.64260006  0.64260006  0.64260006  0.64260006]]

 [[28.20196533 28.20196533 28.20196533 28.20196533 28.20196533
   28.20196533 28.20196533 28.20196533 28.20196533 28.20196533
   28.20196533 28.20196533 28.20196533 28.20196533 28.20196533]]

 [[ 6.5112908   6.51084975  6

In [ ]:
# It looks like the data The data is not formated correctly..
# for some reason, all of the values in a given index (of which the feature measurements shouold be different, are all the same)

In [63]:
print('Training LSTM...')

batch_size = 2 # I think I want to use batch_size = 1


model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          validation_data=(X_test, y_test))

score, acc = model.evaluate(X_test, y_test,

                            batch_size=batch_size)

Training LSTM...
Train on 51180 samples, validate on 17061 samples
Epoch 1/5
32748/51180 [==================>...........] - ETA: 44s - loss: 0.6417 - acc: 0.6527

KeyboardInterrupt: 

In [ ]:
print('score:', score)

print('accuracy:', acc)

In [ ]:
for pattern in dataX:
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(len(alphabet))
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    print(seq_in, "->", result)

In [ ]:
#manager.train_model()

# manager.test_model()
